# DT Assignment

# Data Loading

In [1]:
import pandas as pd 
import numpy as np

In [2]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


## Gini 계수를 구하는 함수 만들기

- Input: df(데이터), label(타겟변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

In [58]:
def get_gini(df, label):
    unique_val = np.unique(df[label])
    
    label0 = (df[label] == unique_val[0]).sum()
    label1 = len(df) - label0
    
    gini = 1 - (label0 / len(df)) ** 2 - (label1 / len(df)) ** 2
    
    return gini

In [59]:
get_gini(pd_data,'class_buys_computer')

0.4591836734693877

## Feature의 Class를 이진 분류로 만들기
- ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})

- Input: df(데이터), attribute(Gini index를 구하고자 하는 변수명)
- Income 변수를 결과로 출력해주세요.

In [8]:
u = np.unique(pd_data['age'])
list(combinations(u,2))

[('middle_aged', 'senior'), ('middle_aged', 'youth'), ('senior', 'youth')]

In [21]:
from itertools import combinations

In [37]:
def get_binary_split(df, attribute):    
    unique_val = np.unique(df[attribute])

    result = []
    for i in range(1,len(unique_val)):
        for j in combinations(unique_val,i):
            result.append(list(j))
            
    return result

In [6]:
get_binary_split(pd_data,"income")

[['high'],
 ['low'],
 ['medium'],
 ['high', 'low'],
 ['high', 'medium'],
 ['low', 'medium']]

## 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 두 함수를 사용하여 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다.

In [104]:
def get_attribute_gini_index(df, attribute, label):    
    split = get_binary_split(df, attribute)
    unique_val = np.unique(df[attribute])

    result = {}

    for s in split:
        name = "_".join(s)
        if len(s) == 1:
            T_data = df[df[attribute]==name]
            F_data = df[df[attribute]!=name]

            gini = (len(T_data) / n) * get_gini(T_data, label) + (len(F_data) / n) * get_gini(F_data, label)

            result[name] = gini
        else:
            result[name] = result[list(set(unique_val) - set(s))[0]]
            
    return result

In [105]:
get_attribute_gini_index(pd_data, "income", "class_buys_computer")

{'high': 0.44285714285714295,
 'low': 0.45,
 'medium': 0.4583333333333333,
 'high_low': 0.4583333333333333,
 'high_medium': 0.45,
 'low_medium': 0.44285714285714295}

- 여기서 가장 작은 Gini index값을 가지는 class를 보여주세요.

In [106]:
min(get_attribute_gini_index(pd_data, "income", "class_buys_computer").items())

('high', 0.44285714285714295)

## 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.
- 모든 변수에 대한 Gini index(최소)를 출력해주세요.
- 해당 결과는 아래와 같이 나와야 합니다.

In [110]:
# 변수명 중 마지막에 위치한 label 컬럼 얻기
label = pd_data.columns[-1]
# label 변수를 제외한 변수명 얻기
features = list(pd_data.columns[:-1])

# 각 변수를 대상으로 반복문 수행(해당 변수 중 가장 낮은 gini 계수와 변수 출력)
for feature in features:
    print('Minimum Gini index of %s :' %feature, min(get_attribute_gini_index(pd_data, feature, label).items())[-1].round(4))

Minimum Gini index of age : 0.3571
Minimum Gini index of income : 0.4429
Minimum Gini index of student : 0.3673
Minimum Gini index of credit_rating : 0.4286


## 위에서 선정한 feature로 DataFrame을 split한 후 나눠진 2개의 DataFrame에서 각각 다음으로 중요한 변수를 선정하고 해당 변수의 Gini index를 제시해주세요.

In [111]:
get_attribute_gini_index(pd_data, 'age', label)


{'middle_aged': 0.35714285714285715,
 'senior': 0.45714285714285713,
 'youth': 0.3936507936507937,
 'middle_aged_senior': 0.3936507936507937,
 'middle_aged_youth': 0.45714285714285713,
 'senior_youth': 0.35714285714285715}

gini가 가장 작은 middle_aged 변수를 가장 중요한 변수로 선정. <br>
middle_aged와 senior_youth로 dataframe을 나눈다. 

In [136]:
midAge_data = pd_data[pd_data['age'] =='middle_aged']
seniorYouth_data = pd_data[pd_data['age'] !='middle_aged']

In [137]:
midAge_data

,age,income,student,credit_rating,class_buys_computer
2,middle_aged,high,no,fair,yes
6,middle_aged,low,yes,excellent,yes
11,middle_aged,medium,no,excellent,yes
12,middle_aged,high,yes,fair,yes


In [138]:
seniorYouth_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes
10,youth,medium,yes,excellent,yes
13,senior,medium,no,excellent,no


In [127]:
# 각 변수를 대상으로 반복문 수행
# features.remove('age')

for feature in features:
    min_gini = min(get_attribute_gini_index(midAge_data, feature, label).items())[-1].round(4)
    print('Age : Middle_aged, Minimum Gini index of %s :' %feature, min_gini)

print("#############################################################")

for feature in features:
    min_gini = min(get_attribute_gini_index(seniorYouth_data, feature, label).items())[-1].round(4)
    print('Age : Middle_aged, Minimum Gini index of %s :' %feature, min_gini)

Age : Middle_aged, Minimum Gini index of income : 0.0
Age : Middle_aged, Minimum Gini index of student : 0.0
Age : Middle_aged, Minimum Gini index of credit_rating : 0.0
#############################################################
Age : Middle_aged, Minimum Gini index of income : 0.2679
Age : Middle_aged, Minimum Gini index of student : 0.2286
Age : Middle_aged, Minimum Gini index of credit_rating : 0.2976


senior_youth 데이터프레임에서는 student 변수가 가장 중요한 변수.

In [31]:
# 각 변수를 대상으로 반복문 수행
features.remove('age')

for feature in features:
    min_gini = min(get_attribute_gini_index(midAge_data, feature, label).items())[-1].round(4))
    print('Age : Middle_aged, Minimum Gini index of %s :' %feature, min_gini)

print("#############################################################")

for feature in features:
    min_gini = min(get_attribute_gini_index(seniorYouth, feature, label).items())[-1].round(4))
    print('Age : Middle_aged, Minimum Gini index of %s :' %feature, min_gini)



Age : Middle_aged, Minimum Gini index of income : 0.0000
Age : Middle_aged, Minimum Gini index of student : 0.0000
Age : Middle_aged, Minimum Gini index of credit_rating : 0.0000
#############################################################
Age : Middle_aged, Minimum Gini index of income : 0.3750
Age : Middle_aged, Minimum Gini index of student : 0.3200
Age : Middle_aged, Minimum Gini index of credit_rating : 0.4167


## Entropy 를 구하는 함수 만들기

In [192]:
def getEntropy(df, feature) :
    entropy = 0
    p = df[feature].value_counts() / len(df)
    
    for i in p:
        entropy -= i * np.log2(i)
    
    return entropy

In [193]:
getEntropy(pd_data, "class_buys_computer")

0.9402859586706311

In [177]:
def getGainA(df, feature) :
    result = {}
    
    info_D = getEntropy(df, feature) # 목표변수에 대한 Entropy 를 구하기
    columns = list(df.loc[:, df.columns != feature]) # 목표변수(feature)를 제외한 나머지 변수들의 변수명을 리스트 형태로 저장
    
    for col in columns:
        entropy = 0
        unique_val = np.unique(df[col])
        print(unique_val)
        
        for val in unique_val:
            val_data = df[df[col].isin([val])]
            entropy += len(val_data) / len(df) * getEntropy(val_data, feature)
    
        result[col] = info_D - entropy
            
    return result

In [194]:
getGainA(pd_data, "class_buys_computer")

{'age': 0.24674981977443933,
 'income': 0.02922256565895487,
 'student': 0.15183550136234159,
 'credit_rating': 0.04812703040826949}